In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
device = 'cpu'

config = {
    'model_name': 'LSTM_2',
    'feature': 'regularized',
    'max_len': 48,
    'min_freq_src': 4,
    'min_freq_trg': 4,
    
    'src_vocab_size': 29798,
    'trg_vocab_size': 21555,

    'embedding_dim': 128,
    'hidden_size': 256,
    'num_layers': 3,

    'num_epochs': 12,
    'weight_decay': 1e-5,
    'label_smoothing': 0.1,
    'dropout': 0.25,

    'learning_rate': 1e-3,
    'gamma': 0.2,
    'patience': 1,
    'threshold': 0.001
}

In [3]:
from dataset import TranslationDataset, Vocab, TrainDataLoader, TestDataLoader
from config import filenames, folders
from lstm2 import LSTM_2

In [4]:
'sds'.isalpha()

True

In [21]:
vocab_src = Vocab(filenames['train_src'], min_freq=config['min_freq_src'])
vocab_trg = Vocab(filenames['train_trg'], min_freq=config['min_freq_trg'])

train_dataset = TranslationDataset(vocab_src, 
                                vocab_trg, 
                                filenames['train_src'], 
                                filenames['train_trg'], 
                                max_len=config['max_len'], 
                                device=device)

val_dataset = TranslationDataset(vocab_src, 
                                vocab_trg, 
                                filenames['test_src'], 
                                filenames['test_trg'], 
                                max_len=72, 
                                device=device, 
                                sort_lengths=False)

100%|██████████| 986/986 [00:00<00:00, 16383.03it/s]


In [22]:
print(len(vocab_src), len(vocab_trg))

29798 21555


In [7]:
unk_idx, pad_idx, bos_idx, eos_idx = 0, 1, 2, 3

train_loader = TrainDataLoader(train_dataset, shuffle=True)
val_loader = TestDataLoader(val_dataset, shuffle=False)

weights_filename = folders['weights'] + 'lstm-save-10.pt'

model = LSTM_2(config=config, weights_filename=weights_filename).to(device)
print(model)

LSTM_2(
  (src_embedding): Embedding(24991, 192)
  (trg_embedding): Embedding(21555, 192)
  (encoder): LSTM(192, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (encoder_output_proj): Linear(in_features=512, out_features=256, bias=True)
  (decoder): LSTM(192, 256, num_layers=3, batch_first=True, dropout=0.2)
  (encoder_hidden_proj): ModuleList(
    (0-2): 3 x Linear(in_features=512, out_features=256, bias=True)
  )
  (encoder_cell_proj): ModuleList(
    (0-2): 3 x Linear(in_features=512, out_features=256, bias=True)
  )
  (fc): Linear(in_features=512, out_features=21555, bias=True)
)


In [8]:
from matplotlib import pyplot as plt
def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Training Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss Over Epochs")
    plt.legend()
    plt.grid()
    plt.show()

In [9]:
model.load(weights_filename)

In [10]:
import numpy as np
np.load('../weights/train.npy')

array([], dtype=float64)

In [11]:
from submission import get_bleu

In [12]:
from dataset import RawDataset
raw_dataset = RawDataset(filenames['test_src'])

In [13]:
model.to(device)
get_bleu(model, val_loader, vocab_trg, filenames, device=device, raw_dataset=raw_dataset)

100%|██████████| 4/4 [00:17<00:00,  4.26s/it]


29.24

In [14]:
# model.save('lstm-deep-cut-vocab-12epoch.pt')

In [17]:
raw_submission = RawDataset(filenames['submission_src'])

In [19]:
from submission import make_submission
from dataset import SubmissionDataset, SubmissionDataLoader
submission_dataset = SubmissionDataset(filenames['submission_src'], vocab_src, device=device)
ldr = SubmissionDataLoader(submission_dataset)
make_submission(model, ldr, vocab_trg, filenames, device=device, raw_dataset=raw_submission)

100%|██████████| 24/24 [00:55<00:00,  2.30s/it]
